<a href="https://colab.research.google.com/github/aliprg0/Google_Colab/blob/main/NasQ_TV_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
from tvDatafeed import TvDatafeed, Interval
import multiprocessing
import time
import glob

In [75]:
def get_nasq_syms():
  with open("watchlist_NASDAQ.txt", "r") as f:
    lines = f.readlines()
    f.close()
  line = lines[0].split(",")
  symbols = []
  for l in line:
    x = l.split(":")
    symbols.append(x[1])
  return symbols
def get_binance_syms():
  with open("binance_tv.txt","r")as f:
    lines = f.readlines()
    f.close()
  nlines = []
  for line in lines:
    line = line.strip()
    nlines.append(line.split(":")[1])
  
  return nlines
def download_data_p(tv,sym):
    data =tv.get_hist(sym,exchange="binance", interval=Interval.in_weekly, n_bars=50000)
    try:
        data.to_csv("data/{}.csv".format(sym))
    except:
      pass  
def download_data(symbols):
  work_with_dir()
  tv = TvDatafeed()
  for sym in symbols:
    print(symbols.index(sym)+1, "/", len(symbols))
    p = multiprocessing.Process(target=download_data_p, name="dd", args=(tv,sym,))
    p.start()
    time.sleep(2)
    p.terminate()
    p.join()

In [83]:
download_data(get_binance_syms())
#download_data(get_nasq_syms())

Data Folder Removed
1 / 574
2 / 574
3 / 574
4 / 574
5 / 574
6 / 574
7 / 574
8 / 574
9 / 574
10 / 574
11 / 574
12 / 574
13 / 574
14 / 574
15 / 574
16 / 574
17 / 574
18 / 574
19 / 574
20 / 574
21 / 574
22 / 574
23 / 574
24 / 574
25 / 574
26 / 574
27 / 574
28 / 574
29 / 574
30 / 574
31 / 574
32 / 574
33 / 574
34 / 574
35 / 574
36 / 574
37 / 574
38 / 574
39 / 574
40 / 574
41 / 574
42 / 574
43 / 574
44 / 574
45 / 574
46 / 574
47 / 574
48 / 574
49 / 574
50 / 574
51 / 574
52 / 574
53 / 574
54 / 574
55 / 574
56 / 574
57 / 574
58 / 574
59 / 574
60 / 574
61 / 574
62 / 574
63 / 574
64 / 574
65 / 574
66 / 574
67 / 574
68 / 574
69 / 574
70 / 574
71 / 574
72 / 574
73 / 574
74 / 574
75 / 574
76 / 574
77 / 574
78 / 574
79 / 574
80 / 574
81 / 574
82 / 574
83 / 574
84 / 574
85 / 574
86 / 574
87 / 574
88 / 574
89 / 574
90 / 574
91 / 574
92 / 574
93 / 574
94 / 574
95 / 574
96 / 574
97 / 574
98 / 574
99 / 574
100 / 574
101 / 574
102 / 574
103 / 574
104 / 574
105 / 574
106 / 574
107 / 574
108 / 574
109 / 57

In [77]:
clmns = [
    "Close 1-2", "Close 2-3", "Close 3-4", "Close 4-5", "Close 5-6", "Close 6-7", "Close 7-8",
    "High 1-2", "High 2-3", "High 3-4", "High 4-5", "High 5-6", "High 6-7", "High 7-8", "High 8-9", "High 9-10", "High 10-11", "High 11-12", "High 12-13", "High 13-14", "High 14-15", "High 15-16",
    "Low 1-2", "Low 2-3", 
    "suggestion"]
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")

  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def read_txt_list():
  with open("yahoo_stocklist.txt","r")as f:
    lines = f.readlines()
    nlines = []
    for line in lines:
       nlines.append(line.strip())
    return nlines
def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["yes","no"], axis=1)
  y = data[["yes","no"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if "symbol" in data.columns:
        data.drop("symbol", axis=1, inplace=True)  
    if "Adj Close" in data.columns:
        data.drop("Adj Close", axis=1, inplace=True)
    if "datetime" in data.columns:
        data.drop("datetime",axis=1,inplace=True)
    if len(pd.DataFrame(data).columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)  

    for i in range(15, data.shape[0]):
        grow = []
      
        s12 = [ 
            data[i-12][0], data[i-12][1], data[i-12][2], data[i-12][3]
        ]
        s11 = [
            data[i-11][0], data[i-11][1], data[i-11][2], data[i-11][3]
        ]
        s10 = [
            data[i-10][0], data[i-10][1], data[i-10][2], data[i-10][3]
        ]
        s9 = [
            data[i-9][0], data[i-9][1], data[i-9][2], data[i-9][3]
        ]
        s8 = [
            data[i-8][0], data[i-8][1], data[i-8][2], data[i-8][3]
        ]
        s7 = [
            data[i-7][0], data[i-7][1], data[i-7][2], data[i-7][3]
        ]
        s6 = [
            data[i-6][0], data[i-6][1], data[i-6][2], data[i-6][3]
        ]
        s5 = [
            data[i-5][0], data[i-5][1], data[i-5][2], data[i-5][3]
        ]   
        s4 = [
            data[i-4][0], data[i-4][1], data[i-4][2], data[i-4][3]
        ]
        s3 = [
            data[i-3][0], data[i-3][1], data[i-3][2], data[i-3][3]
        ]
        s2 = [
            data[i-2][0], data[i-2][1], data[i-2][2], data[i-2][3]
        ]
        s1 = [
            data[i-1][0], data[i-1][1], data[i-1][2], data[i-1][3]
        ]
        

        s1 = scaler(np.array(s1).reshape(-1,1))
        s2 = scaler(np.array(s2).reshape(-1,1))
        s3 = scaler(np.array(s3).reshape(-1,1))
        s4 = scaler(np.array(s4).reshape(-1,1))
        s5 = scaler(np.array(s5).reshape(-1,1))
        s6 = scaler(np.array(s6).reshape(-1,1))
        s7 = scaler(np.array(s7).reshape(-1,1))
        s8 = scaler(np.array(s8).reshape(-1,1))
        s9 = scaler(np.array(s9).reshape(-1,1))
        s10 = scaler(np.array(s10).reshape(-1,1))
        s11 = scaler(np.array(s11).reshape(-1,1))
        s12 = scaler(np.array(s12).reshape(-1,1))
 
        s1 = np.delete(s1, 1)
        s1 = np.delete(s1, 1)
        s2 = np.delete(s2, 1)
        s2 = np.delete(s2, 1)
        s3 = np.delete(s3, 1)
        s3 = np.delete(s3, 1)
        s4 = np.delete(s4, 1)
        s4 = np.delete(s4, 1)
        s5 = np.delete(s5, 1)
        s5 = np.delete(s5, 1)
        s6 = np.delete(s6, 1)
        s6 = np.delete(s6, 1)
        s7 = np.delete(s7, 1)
        s7 = np.delete(s7, 1)
        s8 = np.delete(s8, 1)
        s8 = np.delete(s8, 1)
        s9 = np.delete(s9, 1)
        s9 = np.delete(s9, 1)
        s10 = np.delete(s10, 1)
        s10 = np.delete(s10, 1)
        s11 = np.delete(s11, 1)
        s11 = np.delete(s11, 1)
        s12 = np.delete(s12, 1)
        s12 = np.delete(s12, 1)     

        sugg = "no"
        if data[i][3] > data[i][0]:
            sugg = "yes"

        grow = np.vstack((s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12))

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [78]:
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        unattached_dfs.append(process(pd.read_csv(address)))
     data = np.array(unattached_dfs[0])
     for i in unattached_dfs[1:]:
           data = np.append(data, np.array(i), axis=0)

     unattached_dfs = []
     
     data = pd.DataFrame(data, columns=clmns)
 

     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  print("Done")
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")     

In [ ]:
folder_name = extract_data(100)
to_par(folder_name,10)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

In [85]:
model = Sequential()




model.add(Dense(512, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))


opt = tf.keras.optimizers.Adamax()

model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_86 (Dense)            (None, 512)               12800     
                                                                 
 dense_87 (Dense)            (None, 512)               262656    
                                                                 
 dense_88 (Dense)            (None, 512)               262656    
                                                                 
 dense_89 (Dense)            (None, 512)               262656    
                                                                 
 dense_90 (Dense)            (None, 512)               262656    
                                                                 
 dense_91 (Dense)            (None, 512)               262656    
                                                                 
 dense_92 (Dense)            (None, 512)             

In [86]:
model.fit(xTrain,yTrain,epochs=50,batch_size=32,validation_data=(xTest,yTest))

Epoch 1/50
1728/1728 [==============================] - 34s 20ms/step - loss: 0.6703 - accuracy: 0.5828 - val_loss: 0.6564 - val_accuracy: 0.5958
Epoch 2/50
1728/1728 [==============================] - 33s 19ms/step - loss: 0.6325 - accuracy: 0.6290 - val_loss: 0.6581 - val_accuracy: 0.6164
Epoch 3/50
1728/1728 [==============================] - 33s 19ms/step - loss: 0.6020 - accuracy: 0.6555 - val_loss: 0.6355 - val_accuracy: 0.6257
Epoch 4/50
1728/1728 [==============================] - 33s 19ms/step - loss: 0.5665 - accuracy: 0.6893 - val_loss: 0.6260 - val_accuracy: 0.6407
Epoch 5/50
1728/1728 [==============================] - 33s 19ms/step - loss: 0.5279 - accuracy: 0.7196 - val_loss: 0.6337 - val_accuracy: 0.6443
Epoch 6/50
1728/1728 [==============================] - 33s 19ms/step - loss: 0.4807 - accuracy: 0.7552 - val_loss: 0.6501 - val_accuracy: 0.6491
Epoch 7/50
1728/1728 [==============================] - 33s 19ms/step - loss: 0.4216 - accuracy: 0.7953 - val_loss: 0.6713 -

KeyboardInterrupt: ignored

In [53]:
model.save("monthly_tvcrypto_73_lkola3.h5")

In [73]:
def process_for_prediction(data):
    i = -1
    row = []
    if len(pd.DataFrame(data).columns) == 7:
      data = data.iloc[: , 1:]        
    data = np.array(data)
    grow = []
      
    s12 = [ 
            data[i-12][0], data[i-12][1], data[i-12][2], data[i-12][3]
        ]
    s11 = [
            data[i-11][0], data[i-11][1], data[i-11][2], data[i-11][3]
        ]
    s10 = [
            data[i-10][0], data[i-10][1], data[i-10][2], data[i-10][3]
        ]
    s9 = [
            data[i-9][0], data[i-9][1], data[i-9][2], data[i-9][3]
        ]
    s8 = [
            data[i-8][0], data[i-8][1], data[i-8][2], data[i-8][3]
        ]
    s7 = [
            data[i-7][0], data[i-7][1], data[i-7][2], data[i-7][3]
        ]
    s6 = [
            data[i-6][0], data[i-6][1], data[i-6][2], data[i-6][3]
        ]
    s5 = [
            data[i-5][0], data[i-5][1], data[i-5][2], data[i-5][3]
        ]   
    s4 = [
            data[i-4][0], data[i-4][1], data[i-4][2], data[i-4][3]
        ]
    s3 = [
            data[i-3][0], data[i-3][1], data[i-3][2], data[i-3][3]
        ]
    s2 = [
            data[i-2][0], data[i-2][1], data[i-2][2], data[i-2][3]
        ]
    s1 = [
            data[i-1][0], data[i-1][1], data[i-1][2], data[i-1][3]
        ]
        

    s1 = scaler(np.array(s1).reshape(-1,1))
    s2 = scaler(np.array(s2).reshape(-1,1))
    s3 = scaler(np.array(s3).reshape(-1,1))
    s4 = scaler(np.array(s4).reshape(-1,1))
    s5 = scaler(np.array(s5).reshape(-1,1))
    s6 = scaler(np.array(s6).reshape(-1,1))
    s7 = scaler(np.array(s7).reshape(-1,1))
    s8 = scaler(np.array(s8).reshape(-1,1))
    s9 = scaler(np.array(s9).reshape(-1,1))
    s10 = scaler(np.array(s10).reshape(-1,1))
    s11 = scaler(np.array(s11).reshape(-1,1))
    s12 = scaler(np.array(s12).reshape(-1,1))
    
    s1 = np.delete(s1, 1)
    s1 = np.delete(s1, 1)
    s2 = np.delete(s2, 1)
    s2 = np.delete(s2, 1)
    s3 = np.delete(s3, 1)
    s3 = np.delete(s3, 1)
    s4 = np.delete(s4, 1)
    s4 = np.delete(s4, 1)
    s5 = np.delete(s5, 1)
    s5 = np.delete(s5, 1)
    s6 = np.delete(s6, 1)
    s6 = np.delete(s6, 1)
    s7 = np.delete(s7, 1)
    s7 = np.delete(s7, 1)
    s8 = np.delete(s8, 1)
    s8 = np.delete(s8, 1)
    s9 = np.delete(s9, 1)
    s9 = np.delete(s9, 1)
    s10 = np.delete(s10, 1)
    s10 = np.delete(s10, 1)
    s11 = np.delete(s11, 1)
    s11 = np.delete(s11, 1)
    s12 = np.delete(s12, 1)
    s12 = np.delete(s12, 1)
            
        

    grow = np.vstack((s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12))

    arr = np.array(grow).flatten()
    row.append(arr)

    return np.array(row)
def make_prediction(symbol,timeframe):
  if timeframe == "5m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))    
  if timeframe == "15m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1h":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1d":
    data = yf.download(symbol,period="20d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1wk":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1mo":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "2m":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "30m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "5d":
    data = yf.download(symbol,period="5mo",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "90m":
    data = yf.download(symbol,period="5d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "3mo":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  

In [74]:
make_prediction("btc-usd","1d")

[*********************100%***********************]  1 of 1 completed


array([[0.9015149 , 0.14467987]], dtype=float32)

In [70]:
def process_for_prediction(data):
    i = -1
    row = []
    data.drop("symbol",axis=1,inplace=True)   
    data = np.array(data)
    llst = [0, 1, 2, 3, 4]
    grow = []
      
    s12 = [ 
            data[i-12][0], data[i-12][1], data[i-12][2], data[i-12][3]
        ]
    s11 = [
            data[i-11][0], data[i-11][1], data[i-11][2], data[i-11][3]
        ]
    s10 = [
            data[i-10][0], data[i-10][1], data[i-10][2], data[i-10][3]
        ]
    s9 = [
            data[i-9][0], data[i-9][1], data[i-9][2], data[i-9][3]
        ]
    s8 = [
            data[i-8][0], data[i-8][1], data[i-8][2], data[i-8][3]
        ]
    s7 = [
            data[i-7][0], data[i-7][1], data[i-7][2], data[i-7][3]
        ]
    s6 = [
            data[i-6][0], data[i-6][1], data[i-6][2], data[i-6][3]
        ]
    s5 = [
            data[i-5][0], data[i-5][1], data[i-5][2], data[i-5][3]
        ]   
    s4 = [
            data[i-4][0], data[i-4][1], data[i-4][2], data[i-4][3]
        ]
    s3 = [
            data[i-3][0], data[i-3][1], data[i-3][2], data[i-3][3]
        ]
    s2 = [
            data[i-2][0], data[i-2][1], data[i-2][2], data[i-2][3]
        ]
    s1 = [
            data[i-1][0], data[i-1][1], data[i-1][2], data[i-1][3]
        ]
        

    s1 = scaler(np.array(s1).reshape(-1,1))
    s2 = scaler(np.array(s2).reshape(-1,1))
    s3 = scaler(np.array(s3).reshape(-1,1))
    s4 = scaler(np.array(s4).reshape(-1,1))
    s5 = scaler(np.array(s5).reshape(-1,1))
    s6 = scaler(np.array(s6).reshape(-1,1))
    s7 = scaler(np.array(s7).reshape(-1,1))
    s8 = scaler(np.array(s8).reshape(-1,1))
    s9 = scaler(np.array(s9).reshape(-1,1))
    s10 = scaler(np.array(s10).reshape(-1,1))
    s11 = scaler(np.array(s11).reshape(-1,1))
    s12 = scaler(np.array(s12).reshape(-1,1))
    
    s1 = np.delete(s1, 1)
    s1 = np.delete(s1, 1)
    s2 = np.delete(s2, 1)
    s2 = np.delete(s2, 1)
    s3 = np.delete(s3, 1)
    s3 = np.delete(s3, 1)
    s4 = np.delete(s4, 1)
    s4 = np.delete(s4, 1)
    s5 = np.delete(s5, 1)
    s5 = np.delete(s5, 1)
    s6 = np.delete(s6, 1)
    s6 = np.delete(s6, 1)
    s7 = np.delete(s7, 1)
    s7 = np.delete(s7, 1)
    s8 = np.delete(s8, 1)
    s8 = np.delete(s8, 1)
    s9 = np.delete(s9, 1)
    s9 = np.delete(s9, 1)
    s10 = np.delete(s10, 1)
    s10 = np.delete(s10, 1)
    s11 = np.delete(s11, 1)
    s11 = np.delete(s11, 1)
    s12 = np.delete(s12, 1)
    s12 = np.delete(s12, 1)
        

    grow = np.vstack((s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12))

    arr = np.array(grow).flatten()
    row.append(arr)

    return np.array(row)
def make_prediction(symbol,timeframe):
   tv = TvDatafeed()
   data = tv.get_hist(symbol='btcUSDt',exchange='binance',interval=Interval.in_daily,n_bars=30)
   return model.predict(process_for_prediction(data))

In [71]:
make_prediction("s","x")

array([[0.94422555, 0.09328011]], dtype=float32)

In [72]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
